# LSB Steganography Lab 
**Objetivo:** Generar un dataset de imágenes (ruido) y ocultar un mensaje en **una** de ellas usando LSB.


## Parte 1 — Generador del dataset

Vamos a crear:
- **N = 100** imágenes PNG de **200×200** con ruido aleatorio (RGB).
- Elegiremos **una** imagen al azar y ocultaremos el mensaje:
  `FLAG{WRITE_YOUR_NAME}` (lo cambiaremos por tu nombre)
- No mostraremos cuál es la imagen con mensaje (simula escenario forense real).

Técnica de ocultación:
- Convertimos el mensaje a bits (ASCII).
- Añadimos un **delimitador final** para poder detectar el final al extraerlo.
- Recorremos los canales R,G,B de los píxeles y sustituimos su bit menos significativo (LSB).


# Imports y configuración

In [1]:
# Imports y configuración

from PIL import Image
import numpy as np
import os
import random

# Configuración base del lab
FOLDER = "dataset_images"
NUM_IMAGES = 100
W, H = 200, 200

# Cambia esto por tu nombre
FLAG = "FLAG{LUCIACANTOSBURGOS}"

## Función 1: crear una imagen de ruido

Generamos una matriz `H×W×3` con valores aleatorios 0–255 (uint8),
la convertimos a imagen RGB y la guardamos como PNG.


In [2]:
def create_noise_image(filename, width=200, height=200):
    """
    Crea una imagen RGB de ruido aleatorio y la guarda en PNG.
    """
    arr = np.random.randint(0, 256, size=(height, width, 3), dtype=np.uint8)
    img = Image.fromarray(arr, mode="RGB")
    img.save(filename, format="PNG")

## Función 2: texto a binario (ASCII)

Ejemplo:
- "A" → 65 → `01000001`

Cada carácter → 8 bits.

In [3]:
def text_to_binary(message: str) -> str:
    """
    Convierte un string a bits ASCII (8 bits por carácter).
    """
    return "".join(format(ord(ch), "08b") for ch in message)

## Delimitador de fin

Para saber dónde acaba el mensaje al extraerlo, añadimos un patrón de bits "improbable".
La guía propone: `1111111111111110` (16 bits).

Al reconstruir el texto, cortaremos cuando detectemos este delimitador.

In [4]:
END_DELIMITER = "1111111111111110"  # 16 bits


## Función 3: Ocultar mensaje usando LSB

Idea:
- Abrimos la imagen y obtenemos su array RGB.
- Convertimos el mensaje a bits + delimitador.
- Recorremos los canales de píxeles en orden (R,G,B,R,G,B,...):
  - limpiamos LSB: `value & 0xFE`
  - insertamos bit: `value | bit`
- Guardamos la imagen modificada.


In [5]:
def hide_lsb(image_path: str, message: str):
    """
    Oculta un mensaje en una imagen PNG usando LSB y sobrescribe el archivo.
    """
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)

    bits = text_to_binary(message) + END_DELIMITER
    flat = data.reshape(-1)  # aplana todos los canales en un vector

    if len(bits) > len(flat):
        raise ValueError("El mensaje es demasiado largo para la capacidad de la imagen.")

    # Inserción LSB
    for i, b in enumerate(bits):
        flat[i] = (flat[i] & 0xFE) | int(b)

    # Reconstruimos y guardamos
    stego = flat.reshape(data.shape)
    Image.fromarray(stego, mode="RGB").save(image_path, format="PNG")


## Función 4: generar dataset

Pasos:
1. Crear carpeta si no existe.
2. Generar N imágenes de ruido.
3. Elegir una al azar.
4. Ocultar el mensaje en esa imagen.
5. **No imprimir** cuál era (importante para la práctica).


In [6]:
def generate_dataset(folder: str, num_images: int, secret_message: str, width=200, height=200):
    os.makedirs(folder, exist_ok=True)

    filenames = []
    for i in range(num_images):
        fname = os.path.join(folder, f"img_{i:03d}.png")
        create_noise_image(fname, width=width, height=height)
        filenames.append(fname)

    secret_file = random.choice(filenames)
    hide_lsb(secret_file, secret_message)


## Ejecutar Parte 1

Esto creará `dataset_images/` con 100 PNG.
Una de ellas contiene el mensaje oculto (no sabemos cuál).


In [7]:
generate_dataset(FOLDER, NUM_IMAGES, FLAG, width=W, height=H)
print(f"Dataset creado: {NUM_IMAGES} imágenes en '{FOLDER}/'")
print("Una de ellas contiene un mensaje oculto. Good luck!")


Dataset creado: 100 imágenes en 'dataset_images/'
Una de ellas contiene un mensaje oculto. Good luck!
